In [ ]:
#Group names: Neel, Julius, Sahar, Anna
#Imported Libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import string
import dbm
import pickle

In [ ]:
#pip installs
!pip3 install ipython-sql --user
!pip3 install mysql-connector --user
!pip3 install mysql --user

In [ ]:
#organize database 

import mysql
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="nsangani",
  passwd="nsangani",
  database="nsangani_db"
)

#'cursor' object to run commands
mycursor = mydb.cursor(buffered=True)

In [ ]:
mycursor.execute('SELECT * FROM Organizations')
Organizations = mycursor.fetchall() 

mycursor.execute('SELECT * FROM Modalities')
Modalities = mycursor.fetchall() 

mycursor.execute('SELECT * FROM Exam_to_cpt')
Exam_to_cpt = mycursor.fetchall() 

mycursor.execute('SELECT * FROM Studies')
Studies = mycursor.fetchall() 

mycursor.execute('SELECT * FROM Case_Log')
case_log = mycursor.fetchall() 


In [ ]:
                                                #### END OF PART 1 #####

In [ ]:
# Dictionary of defined case_log_categories:cpt

d=dict()

d['XR'] = ['71045','71046','71047','71048',]
d['CT'] = ['72192', '72193', '72194', '74150', '74160', '74170', '74176', '74177', '74178',]
d['CTA'] = ['70496', '70498', '70544', '70545', '70546', '70547', '70548', '70549', '71275', '71555', '72159', '72191', '72198', '73206', '73225', '73725', '73706', '74174', '74175', '74185',] 
d['Image'] = ['20604', '20606', '20611', '32555', '32557', '49083', '49405', '49406', '49407', '77012', '76942',]
d['Mammography'] = ['77065', '77066', '77067',] 
d['MRI_Body'] = ['71550','71551','71552','72195','72196','72197','74181','74182','74183','74712','74713',] 
d['MRI_Brain'] = ['70551','70552','70553',] 
d['MRI_Joints'] = ['73721','73722','73723',]
d['MRI_Spine'] = ['72141','72142', '72146', '72147', '72148', '72149', '72156', '72157', '72158',]
d['PET'] = ['78459', '78491', '78492', '78608', '78609', '78811', '78812', '78813', '78814', '78815', '78816',]
d['US'] = ['76700', '76705', '76706', '76770', '76775', '76830', '76856', '76857',]


In [ ]:
# Dictionary of Resident:exam_code after extracting both using nested search

mycursor.execute('SELECT trainee_name, exam_code FROM Studies WHERE exam_code IN (select exam_code From Exam_to_cpt Where cpt IN (Select cpt from Case_Log));')
tab = mycursor.fetchall()
#print(tab)

rt={}
for i in tab:
    r=i[0]
    e=i[1]
    if r not in rt:
        rt[r]=[e]
    else:
        rt[r].append(e)
# print(rt)

In [ ]:
# Making exam_code:cpt dictionary using the defined cpt values using above case_log_categories:cpt dictionary

x=Exam_to_cpt 

case_study=dict()
for i in x:
    n=0
    exam=i[1]  #exam_code column
    cpt=i[2]   #cpt column from Exam_to_cpt
    for k,v in d.items(): #calling dict 'd' from above when 'key' is case_log_categories and 'values' is cpt given
        if str(cpt) in v: 
            case_study[exam]=k
# print(case_study)

In [ ]:
# Combining exam_code:cpt and resident:cpt dictionaries to get the counter 
# for each case_log_categories in respect to each resident

def call_resident_count(resident):
    XR = 0
    CT = 0
    CTA = 0
    Image = 0
    Mammography = 0
    MRI_Body = 0
    MRI_Brain = 0
    MRI_Joints = 0
    MRI_Spine = 0
    PET = 0
    US = 0
    dic=dict()
    v = rt.get(resident)
    case = []
    for i in v:
        g = case_study.get(i)
        case.append(g)
    for i in case:
        if i not in dic:
            dic[i]=1
        else:
            dic[i]+=1
    return (dic)

# print(call_resident_count('Resident21'))

for i in rt:
    call_resident_count(i)

In [ ]:
# Inserting Resident stats to PhpMyAdmin using the call_resident_count function defined above

r='residents'
for i in rt:
    forbidden=[]
    res=call_resident_count(i)
    for k,v in res.items():
        if i not in forbidden:
            woomy="INSERT INTO resident_stat(%s,%s) VALUES('%s',%s)" %(r,k,i,v)
            mycursor.execute(woomy)
            mydb.commit()
            forbidden.append(i)
        else:
            woomy="UPDATE resident_stat SET %s=%s WHERE %s='%s'" %(k,v,r,i)
            mycursor.execute(woomy)
            mydb.commit()

In [ ]:
# THIS PART WAS DONE USING REGULAR PYTHON IDLE 
# IMPORTED LIBRARY IS MENTIONED ABOVE FOR THIS PART AS WELL

# Result from this was saved on excel and then inserted into PhpMyAdmin using database query

# Two tables were made: Division and condition_met

#Division, as mentioned in previous cell, is percent exam finished out of required minimum exam in each Case_log_categories for each resident 
#condition_met is the values replaced with 'MET' if they met the minimum exam requirement 

# db=dbm.open('database','c')
# x=open('resident_stats.csv')
# n=0
# dic=dict()
# d=[]
# for i in x:
#     if n==0:
#         n=1
#         b=i.split(',')
#         for i in b:
#             d.append(i)
#         print(d)
#     else:
#         b=i.split(',')
#         for ii in b:
#             if ii==b[0]:
#                 dd=int(ii)
#                 divide=dd/1900

#                 else: 
#                     d[0] = 1900 - dd

#             elif ii==b[1]:
#                 dd=int(ii)
#                 divide=dd/600

#                 else: 
#                     d[1] = 600 - dd
                    
#             elif ii==b[2]:
#                 dd=int(ii)
#                 divide=dd/100

#                 else: 
#                     d[2] = 100 - dd
                    
#             elif ii==b[3]:
#                 dd=int(ii)
#                 divide=dd/25

#                 else: 
#                     d[3] = 25 - dd
                    
#             elif ii==b[4]:
#                 dd=int(ii)
#                 divide=dd/300

#                 else: 
#                     d[4] = 300 - dd
                    
#             elif ii==b[5]:
#                 dd=int(ii)
#                 divide=dd/20

#                 else: 
#                     d[5] = 20 - dd
                    
#             elif ii==b[6]:
#                 dd=int(ii)
#                 divide=dd/110

#                 else: 
#                     d[6] = 110 - dd
                    
#             elif ii==b[7]:
#                 dd=int(ii)
#                 divide=dd/60

#                 else: 
#                     d[7] = 60 - dd
                    
#             elif ii==b[8]:
#                 dd=int(ii)
#                 divide=dd/30

#                 else: 
#                     d[8] = 30 - dd
                    
#             elif ii==b[9]:
#                 dd=int(ii)
#                 divide=dd/350

#                 else: 
#                     d[9] = 350 - dd
                
#         print(dic)
#         p=pickle.dumps(dic)
#         db[b[11]]=p
#         dic=dict()
# db.close() 

In [ ]:
#Percent exam finished out of required minimum exam in each Case_log_categories for each resident 

mycursor.execute('SELECT * FROM division')
division = mycursor.fetchall() 
print (division)

In [ ]:
# THIS PART WAS DONE USING REGULAR PYTHON IDLE 
# IMPORTED LIBRARY IS MENTIONED ABOVE FOR THIS PART AS WELL

# Two tables were made: Division and condition_met

#Division, as mentioned in previous cell, is percent exam finished out of required minimum exam in each Case_log_categories for each resident 
#condition_met is the values replaced with 'MET' if they met the minimum exam requirement 

# db=dbm.open('database','c')
# x=open('resident_stats.csv')
# n=0
# dic=dict()
# d=[]
# for i in x:
#     if n==0:
#         n=1
#         b=i.split(',')
#         for i in b:
#             d.append(i)
#         print(d)
#     else:
#         b=i.split(',')
#         for ii in b:
#             if ii==b[0]:
#                 dd=int(ii)
#                 divide=dd/1900
#                 if (divide*100) >= 100:
#                     d[0]= 'Met'
#                 else: 
#                     d[0] = 1900 - dd

#             elif ii==b[1]:
#                 dd=int(ii)
#                 divide=dd/600
#                 if (divide*100) >= 100:
#                     d[1]= 'Met'
#                 else: 
#                     d[1] = 600 - dd
                    
#             elif ii==b[2]:
#                 dd=int(ii)
#                 divide=dd/100
#                 if (divide*100) >= 100:
#                     d[2]= 'Met'
#                 else: 
#                     d[2] = 100 - dd
                    
#             elif ii==b[3]:
#                 dd=int(ii)
#                 divide=dd/25
#                 if (divide*100) >= 100:
#                     d[3]= 'Met'
#                 else: 
#                     d[3] = 25 - dd
                    
#             elif ii==b[4]:
#                 dd=int(ii)
#                 divide=dd/300
#                 if (divide*100) >= 100:
#                     d[4]= 'Met'
#                 else: 
#                     d[4] = 300 - dd
                    
#             elif ii==b[5]:
#                 dd=int(ii)
#                 divide=dd/20
#                 if (divide*100) >= 100:
#                     d[5]= 'Met'
#                 else: 
#                     d[5] = 20 - dd
                    
#             elif ii==b[6]:
#                 dd=int(ii)
#                 divide=dd/110
#                 if (divide*100) >= 100:
#                     d[6]= 'Met'
#                 else: 
#                     d[6] = 110 - dd
                    
#             elif ii==b[7]:
#                 dd=int(ii)
#                 divide=dd/60
#                 if (divide*100) >= 100:
#                     d[7]= 'Met'
#                 else: 
#                     d[7] = 60 - dd
                    
#             elif ii==b[8]:
#                 dd=int(ii)
#                 divide=dd/30
#                 if (divide*100) >= 100:
#                     d[8]= 'Met'
#                 else: 
#                     d[8] = 30 - dd
                    
#             elif ii==b[9]:
#                 dd=int(ii)
#                 divide=dd/350
#                 if (divide*100) >= 100:
#                     d[9]= 'Met'
#                 else: 
#                     d[9] = 350 - dd
                
#         print(dic)
#         p=pickle.dumps(dic)
#         db[b[11]]=p
#         dic=dict()
# db.close() 

In [ ]:
#Replace 100% or 1.0 or higher to 'MET' for all the resident if they met the minimum exam requirement

mycursor.execute('SELECT * FROM condition_met')
condition_met = mycursor.fetchall() 

In [ ]:
#Bullet one: case log categories : total v remaining
#Graph each Case Log Category total completed vs remaining for the entire group overall. 
#Each Case Log Category should be represented separately on your graph.

select = "SELECT exam_code FROM Studies where exam_code IN (select exam_code FROM Exam_to_cpt)" 
mycursor.execute(select)
exam = mycursor.fetchall()

select = "SELECT exam_code, cpt FROM Exam_to_cpt" 
mycursor.execute(select)
cpt = mycursor.fetchall()

#These keys and values come from the pdf 
caseLog = {}
caseLog["Chest_X_ray"] = "71045", "71046", "71047", "71048"
caseLog["CT_Abd_Pel"] = "72192", "72193", "72194", "74150", "74160", "74170", "74176", "74177", "74178"
caseLog["CTA_MRA"] = "70496", "70498", "70544", "70545", "70546","70547", "70548", "70549", "71275", "71555", "72159","72191", "72198", "73206", "73225", "73725", "73706","74174", "74175", "74185"
caseLog["Image_Guided_Bx"] = "20604", "20606", "20611", "32555", "32557","49083", "49405", "49406", "49407", "77012", "76942"
caseLog["Mammography"] = "77065", "77066", "77067"
caseLog["MRI_Body"] = "71550", "71551", "71552", "72195", "72196", "72197", "74181", "74182", "74183", "74712", "74713" 
caseLog["MRI_Brain"] = "70551", "70552", "70553" 
caseLog["MRI_Lower_Extremity"] = "73721", "73722", "73723" 
caseLog["MRI_Spine"] = "72141", "72142", "72146", "72147", "72148", "72149", "72156", "72157", "72158"
caseLog["PET"] = "78459", "78491", "78492", "78608", "78609","78811", "78812", "78813", "78814", "78815", "78816"
caseLog["US_ABD_Pel"] = "76700", "76705", "76706", "76770", "76775", "76830","76856", "76857"

casel_num = dict()

count = []
#Logic: if exam code is equal in both exam and cpt, cpt is replaced with case log category and new list is created [exam code, case log category]
for e in exam:
    for x in cpt:
        if e[0] in x[0]:
            for k in caseLog:
                if x[1] in caseLog[k]:
                    new = [e[0],k]
                    count.append(new)     

case_count = dict()
#We loop through the list count, and put the datas in dictionary case_count - keys are case log categories, and values are exam codes
for i in count:
    if i[1] not in case_count:
        case_count[i[1]] = [i[0]]
    else:
        case_count[i[1]].append(i[0])

    

barWidth = 0.15

# Remaining comes from pdg minimum requirement values
#Complete is divided by 25 to account for 25 residents
complete = [0, 814/25, 1011/25, (len(case_count['Image_Guided_Bx'])/25), 0, (len(case_count['MRI_Body'])/25), (len(case_count['MRI_Brain'])/25),(len(case_count['MRI_Lower_Extremity'])/25),(len(case_count['MRI_Spine'])/25), 0,(len(case_count['US_ABD_Pel'])/25)]
remaining = [1900, 600, 100, 25-(len(case_count['Image_Guided_Bx'])/25), 300, 20-(len(case_count['MRI_Body'])/25), 110-(len(case_count['MRI_Brain'])/25), 20-(len(case_count['MRI_Lower_Extremity'])/25),60-(len(case_count['MRI_Spine'])/25), 30, 350-(len(case_count['US_ABD_Pel'])/25)]


r1 = np.arange(len(complete))
r2 = [x + barWidth for x in r1]

plt.rcParams['xtick.labelsize']=12
plt.rcParams["figure.figsize"] = (20,30)
plt.yticks()


plt.bar(r1, complete, color='#3333dd', width=barWidth, edgecolor='white', label='Complete')
plt.bar(r2, remaining, color='#dd3333', width=barWidth, edgecolor='white', label='Remain')
plt.title('Completed Vs Remaining Exams among 25 residents') 
plt.ylabel('Minimum Exams')
plt.xlabel('Case Log Categories')
plt.xticks([r + barWidth for r in range(len(complete))], ['Chest X-ray', 'CT Abd/Pel', 'CTA/MRA', 'Img Bx', 'Mammography', 'MRI Body','MRI Brain', 'MRI Lower','MRI Spine', 'PET', 'US Abd/Pel'])

plt.legend()
plt.show()

In [ ]:
#Part Two - Imcomplete RVUS 
fig, ax = plt.subplots() 
select = "SELECT trainee_name, exam_code FROM Studies where exam_code IN (select exam_code FROM Exam_to_cpt)" 
mycursor.execute(select)
c = mycursor.fetchall()
# C is exam code and trainee name
select = "SELECT exam_code, rvu From Exam_to_cpt"
mycursor.execute(select)
s = mycursor.fetchall()
#S is exam code and rvu
d = dict()
for i in s:
    #when exam code is equal in c and s, dict[trainee name] = RVU
    for a in c:
        if i[0] == a[1]:
            if a[0] not in d:
                d[a[0]] = [str(i[1])]
            else:
                d[a[0]].append(str(i[1]))

# count the occurrence of each class 
# get x and y data 


r = dict()
#Dictionary that will become x y values in graph


#Iterate through dict, and if value is none or zero we count it with var i
i = 0
c=0
for x in d:
    c = c + 1
    for v in d[x]:
        if (v == 'None')or(v=='0'):
            i += 1
        r[c] = i

print(r)

ax.bar(r.keys(),r.values()) 
# set title and labels 
ax.set_title('RVU incomplete') 
ax.set_xlabel('residents') 
ax.set_ylabel('amount')

In [ ]:
#Part Two RVUS completed
fig, ax = plt.subplots() 
select = "SELECT trainee_name, exam_code FROM Studies where exam_code IN (select exam_code FROM Exam_to_cpt)" 
mycursor.execute(select)
c = mycursor.fetchall()
# C is exam code and trainee name
select = "SELECT exam_code, rvu From Exam_to_cpt"
mycursor.execute(select)
s = mycursor.fetchall()
#S is exam code and rvu
d = dict()
for i in s:
    #when exam code is equal in c and s, dict[trainee name] = RVU
    for a in c:
        if i[0] == a[1]:
            if a[0] not in d:
                d[a[0]] = [str(i[1])]
            else:
                d[a[0]].append(str(i[1]))

# count the occurrence of each class 
# get x and y data 

r = dict()
# Dict keys and values become x and y in graph - used len function to get amount for each resident

i = 0
c=0
for x in d:
    c = c + 1
    for v in d[x]:
        if (v != 'None')and(v!='0'):
            i += 1
        r[c] = i

ax.bar(r.keys(),r.values()) 
# set title and labels 
ax.set_title('RVU Completed') 
ax.set_xlabel('residents') 
ax.set_ylabel('amount complete')

In [ ]:
#Part three of part three
select = "Select organization_code,organization_id from Organizations"
mycursor.execute(select)
org = mycursor.fetchall()

select = "Select exam_code,organization_id from Studies"
mycursor.execute(select)
stu = mycursor.fetchall()

select = "Select exam_code,rvu from Exam_to_cpt"
mycursor.execute(select)
exam_cpt = mycursor.fetchall()
fig, ax = plt.subplots() 
counter = {}
nones_count = {}

#My logic here: if organization id is equal in org and stu and exam code is equal in stu- if exam code is equal in stu and exam_cpt then dic value is added d[org_code] = rvu
d = {}
for o in org:
    for s in stu:
        if o[1] == s[1]:
            for e in exam_cpt:
                if s[0] == e[0]:
                    if o[0] not in d:
                        d[o[0]] = [e[1]]
                    else:
                        d[o[0]].append(e[1])

#Counting thr rvus
for k in d:
    one = 0
    two = 0
    for item in d[k]:
        if item == 1 or item == 2 or item == 3:
            one = one + 1
        else:
            two = two + 1
    counter[k] = one
    nones_count[k] = two
    

barWidth = 0.30

nones = nones_count.values()
required = counter.values()


r1 = np.arange(len(nones))
r2 = [x + barWidth for x in r1]


plt.rc('axes', titlesize=22)     
plt.rc('axes', labelsize=22)    
plt.rc('xtick', labelsize=22)    
plt.rc('ytick', labelsize=22)
plt.rcParams['xtick.labelsize']=22
plt.rcParams["figure.figsize"] = (55, 20)
plt.yticks()


plt.bar(r1, nones, color='#3333dd', width=barWidth, edgecolor='white', label='nones')
plt.bar(r2, required, color='#dd3333', width=barWidth, edgecolor='white', label='required')
plt.title('Required/non-required by organization code') 
plt.ylabel('amount')
plt.xlabel('Organizations')

xticks = []
for i in nones_count.keys():
    xticks.append(i)
plt.xticks([r + barWidth for r in range(len(nones))], xticks)

plt.legend()
plt.show()    
